# ABD Patent Cümlesinden Cümle Eşleştirmesine--U.S. Patent Phrase to Phrase Matching

Bu proje, belirli kelime çiftlerinin (bir "anchor" ve bir "target" kelime) benzerliklerini değerlendirmenizi istemektedir. Değerlendirme, 0'dan (hiç benzer değil) 1'e (tamamen aynı anlama geliyor) kadar olan bir ölçekte yapılmalıdır. Ancak bu, tipik bir semantik benzerlik görevinden farklıdır çünkü benzerlik burada bir patent bağlamı içinde değerlendirilmiştir. Özellikle, bu patentlerin 2021.05 tarihli CPC sınıflandırmasına (Classification and Patent Classification) göre değerlendirilmiştir. CPC kodları, patentin neyle ilgili olduğunu gösteren sınıflandırmalardır. Örneğin, "kuş" ve "Cape Cod" ifadeleri normal dilde düşük semantik benzerliğe sahip olabilir, ancak "ev" bağlamında düşünüldüğünde anlamları daha yakın olabilir.

Proje, bir kod yarışmasıdır ve gönderilen kodlar, görünmeyen bir test veri kümesi üzerinde çalıştırılacaktır. Bu test verisi yaklaşık 12.000 kelime çifti içerir. Küçük bir halka açık test veri kümesi, test amaçları için sunulmuş olsa da puanlama işleminde kullanılmaz.

Proje dosyaları şunları içerir:
- `train.csv`: Eğitim veri setini içerir ve kelime çiftlerini, bağlamlarını ve benzerlik puanlarını içerir.
- `test.csv`: Eğitim veri seti ile aynı yapıya sahip olan test veri setini içerir, ancak puanı içermez.
- `sample_submission.csv`: Doğru formatta örnek bir gönderim dosyasını içerir.

Proje, kelime çiftlerinin benzerliklerini değerlendirme ve bu benzerlikleri belirleme becerinizi test etmek için tasarlanmıştır. Projeden beklenen, belirli bir kelime çiftinin ne kadar benzer olduğunu değerlendirmek ve benzerlik puanlarını tahmin etmek için bir model veya kod geliştirmenizdir. Sonuçları bir CSV dosyasına kaydedip sunmanız gerekir. Bu, doğal dil işleme (NLP) ve kelime vektörleri gibi metin madenciliği yöntemlerini içerebilecek bir makine öğrenimi uygulamasıdır. Proje, kelime benzerliği ve belirli bir bağlam içinde anlamın nasıl değerlendirileceği konusunda önemli bir sorunu ele almaktadır.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

Tipik bir semantik benzerlik görevi, metinlerin anlamsal benzerliklerini ölçmeyi amaçlar. Metinler arasındaki anlam benzerliği, genellikle kelime vektörleri (word embeddings) veya özellik mühendisliği yöntemleri kullanılarak hesaplanır. Aşağıda tipik bir semantik benzerlik görevinin temel adımları ve yöntemleri açıklanmaktadır:

1. Kelime Gömme (Word Embedding) Kullanımı: Metin benzerliği görevleri için kelime gömme modelleri kullanılır. Bu modeller, her kelimenin metindeki bağlamını temsil eden vektörler oluşturur. Öne çıkan kelime gömme modelleri arasında Word2Vec, GloVe ve FastText bulunur. Bu modeller, kelime benzerliğini hesaplamak için kelime vektörlerini kullanır.

2. Cümlenin Temsil Edilmesi: Metin benzerliği görevi için iki metin arasındaki benzerliği hesaplamak için her iki metni de bir vektörle temsil etmek gerekebilir. Bu işlem için çeşitli yöntemler kullanılabilir. Örneğin, cümlelerin kelime gömme vektörlerinin ortalaması alınabilir veya daha gelişmiş modeller kullanılabilir.

3. Benzerlik Skoru Hesaplama: İki metin arasındaki benzerliği hesaplamak için farklı metrikler kullanılabilir. Öklidyen mesafe, kosinüs benzerliği ve Jaccard benzerliği gibi metrikler sıkça kullanılır. Bu metrikler, iki metin arasındaki anlam benzerliğini ölçmeye yardımcı olur.

4. Eğitim Verileri Kullanma: Semantik benzerlik modelini eğitmek için bir eğitim veri kümesi gereklidir. Bu veri kümesi, benzerlik puanları içeren metin çiftleri veya sınıflandırma etiketleri içerebilir. Model, bu verileri kullanarak metin benzerliğini öğrenir.

5. Değerlendirme: Modelin performansı, genellikle ölçeklendirilmiş benzerlik puanlarına veya sınıflandırma sonuçlarına dayalı olarak değerlendirilir. Bu, modelin ne kadar iyi bir şekilde metin benzerliğini tahmin ettiğini gösterir.

Bu projede temel amaç, metinler arasındaki anlam benzerliğini değerlendirmektir. Ancak, bu görev tipik bir semantik benzerlik görevinden farklıdır çünkü metinlerin benzerliği, özel bir sınıflandırma sistemi olan "CPC (Cooperative Patent Classification)" kodlarına dayalı olarak değerlendirilir. CPC, patentlerin belirli konu alanlarına atanmasını sağlayan bir sınıflandırma sistemidir ve her konu alanı belirli bir kod ile temsil edilir. Bu projede, verilen metin çiftlerinin ne ölçüde benzer olduğunu değerlendirmek için metinlerin CPC kodlarına göre benzerliği ölçülür.

Örneğin, iki metin arasındaki anlam normal bir dilde farklı olabilir, ancak bu metinlerin belirli bir CPC koduna ait bir patent konusu ile ilgili olması durumunda benzerlikleri daha yüksek olabilir. Özellikle patent başvuruları veya patent incelemeleri sırasında, belirli bir patent sınıflandırma kodu (CPC) ile ilgili olan metinlerin ne ölçüde benzer olduğunu değerlendirmek, patent ofisleri ve araştırmacılar için önemlidir. Bu nedenle, bu projede metin benzerliği, patent bağlamında özelleştirilmiş bir yaklaşımla değerlendirilir.

Bu tür projeler, özellikle metin madenciliği, hukuk, patent incelemesi ve teknik yazıların analizi gibi alanlarda uygulanabilir. CPC kodlarına dayalı metin benzerliği değerlendirmesi, bu alanlarda farklı patentler veya belgeler arasındaki benzerlikleri anlamak için kullanılabilir ve bu sayede araştırmacılar veya inceleme uzmanları ilgili belgelere daha hızlı ve etkili bir şekilde erişebilirler.

In [2]:
#pip install textacy

In [3]:
from textacy import preprocessing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
import gensim

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

In [5]:
# Verileri yükle
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
# Veri yapısını incele
print(train_df.head())

                 id     anchor                  target context  score
0  37d61fd2272659b1  abatement  abatement of pollution     A47   0.50
1  7b9652b17b68b7a4  abatement          act of abating     A47   0.75
2  36d72442aefd8232  abatement         active catalyst     A47   0.25
3  5296b0c19e1ce60e  abatement     eliminating process     A47   0.50
4  54c1e3b9184cb5b6  abatement           forest region     A47   0.00


In [6]:
train_df.sample(9)

,id,anchor,target,context,score
6811,310cd85bdd5f068f,conductor particles,copper particles,B01,0.50
23800,392a76a87cf23501,pendent,pendent groups,A43,0.50
6857,0bcd5db179b126e8,conductor particles,conductivity inorganic agent,H01,0.50
24021,c795af6c9f37370f,perform desired action,desired face,B23,0.00
14914,290f0ef44f50b6d8,gripping layer,strength,G03,0.25
9970,62d3ae8be400f811,dna strand,standard deviation,C07,0.00
28845,edd3044721567d2b,rotor stages,rotors stages,F04,1.00
15190,555f7fc2ff305dfe,hardware blocks,hardware data,G06,0.50
3670,b89cd4b4f331a181,boom hydraulic cylinder,boom cylinder,E02,0.50


Bu veri, bir "anchor" ifade ile bir "target" ifade arasındaki benzerliği bir "context" (CPC sınıflandırma) bağlamında değerlendiren örnekler içerir. Ayrıca her örneğin bir benzerlik "score" puanını içerir. İşte her bir sütunun açıklaması:

- "id": Her örneğin benzersiz bir kimliğini temsil eder.
- "anchor": Birinci ifade veya "anchor" ifadesi.
- "target": İkinci ifade veya "target" ifadesi.
- "context": İki ifade arasındaki benzerliği değerlendirmek için kullanılan özel bir sınıflandırma sistemi olan CPC (Cooperative Patent Classification) sınıflandırmasının kodunu içerir. Bu, ifadelerin birbirine olan benzerliğini belirleme bağlamını sağlar.
- "score": İki ifade arasındaki benzerliği değerlendiren bir puan. Puanlar 0 ile 1 arasında değişir ve aşağıdaki anlamlara gelir:
  - 1.0: Çok yakın eşleşme. Genellikle tam bir eşleşme olur, ancak çekim farklılıkları, miktar (tekil ve çoğul gibi), ve stopwords eklemek veya çıkarmak (örneğin, "the", "and", "or") gibi farklılıklar olabilir.
  - 0.75: Yakın eşanlam. Örneğin, "mobil telefon" ve "cep telefonu" gibi. Bu ayrıca kısaltmaları da içerir, örneğin "TCP" -> "transmission control protocol".
  - 0.5: Anlamı aynı olmayan eşanlam. Bu, aynı işlevi veya aynı özellikleri paylaşmayan eşanlamları içerir. Bu aynı zamanda geniş-dar (hyponym) ve dar-geniş (hypernym) eşleşmeleri içerir.
  - 0.25: Bir şekilde ilişkilendirilmiş. İki ifade aynı yüksek seviye alanın içinde olabilir, ancak eşanlamlar değildir. Bu aynı zamanda zıt anlamları içerir.
  - 0.0: İlişkilendirilmemiş. İki ifade arasında herhangi bir benzerlik yok.

Bu veri, bu özel bağlamdaki ifadelerin benzerliğini değerlendirmek için kullanılır ve CPC sınıflandırmasına dayalı olarak ifadelerin anlam benzerliğini ölçer. Projede, "score" puanını tahmin etmek amacıyla bir model oluşturmamız gerekiyor.

In [7]:
train_df.shape

(36473, 5)

In [8]:
train_df["anchor"].value_counts()

component composite coating              152
sheet supply roller                      150
source voltage                           140
perfluoroalkyl group                     136
el display                               135
                                        ... 
plug nozzle                                2
shannon                                    2
dry coating composition1                   2
peripheral nervous system stimulation      1
conduct conducting material                1
Name: anchor, Length: 733, dtype: int64

In [9]:
train_df["target"].value_counts()

composition                    24
data                           22
metal                          22
motor                          22
assembly                       21
                               ..
switching switch over valve     1
switching switch off valve      1
switching over valve            1
switching off valve             1
wooden substrate                1
Name: target, Length: 29340, dtype: int64

In [10]:
train_df.drop(columns = ['id', 'context'], inplace = True)

In [11]:
train_df.head()

,anchor,target,score
0,abatement,abatement of pollution,0.50
1,abatement,act of abating,0.75
2,abatement,active catalyst,0.25
3,abatement,eliminating process,0.50
4,abatement,forest region,0.00


In [12]:
#küçük harfe çevir
train_df["anchor"] = train_df["anchor"].str.lower()
train_df["target"] = train_df["target"].str.lower()

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
#stopwordsleri çıkar
stop = stopwords.words('english')

train_df["anchor"] = train_df['anchor'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train_df["target"] = train_df['target'].apply(lambda y: ' '.join([word for word in y.split() if word not in (stop)]))

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
# Ekleri Kaldırma ve Kökleri Bulma (Lemma & Stemma)
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
train_df['lemmatize_a']=[[lemmatizer.lemmatize(str(i)) for i in words] for words in train_df['anchor'].values]
train_df['lemmatize_t']=[[lemmatizer.lemmatize(str(i)) for i in words] for words in train_df['target'].values]

In [17]:
stemmer = SnowballStemmer("english")
train_df['anchor'] = train_df['anchor'].apply(lambda x: stemmer.stem(x))
train_df['target'] = train_df['target'].apply(lambda x: stemmer.stem(x))

In [18]:
#tokenize
from nltk.tokenize import TreebankWordTokenizer
tokenizer=TreebankWordTokenizer()
train_df['tokenized_a']=[tokenizer.tokenize(i) for i in train_df['anchor'].values]
train_df['tokenized_t']=[tokenizer.tokenize(i) for i in train_df['target'].values]

In [19]:
tf = TfidfVectorizer(analyzer = 'char')

anchor_tf = tf.fit_transform(train_df['anchor']).toarray()
target_tf = tf.transform(train_df['target']).toarray()

In [20]:
train_df.sample(7)

,anchor,target,score,lemmatize_a,lemmatize_t,tokenized_a,tokenized_t
28346,roll workpiec,hot steel rol,0.25,"[r, o, l, l, , w, o, r, k, p, i, e, c, e]","[h, o, t, , s, t, e, e, l, , r, o, l, l, e, r]","[roll, workpiec]","[hot, steel, rol]"
26536,pulping apparatus,fruit pulp,0.00,"[p, u, l, p, i, n, g, , a, p, p, a, r, a, t, ...","[f, r, u, i, t, , p, u, l, p]","[pulping, apparatus]","[fruit, pulp]"
17509,invertebr,cold blooded anim,0.75,"[i, n, v, e, r, t, e, b, r, a, t, e]","[c, o, l, d, , b, l, o, o, d, e, d, , a, n, ...",[invertebr],"[cold, blooded, anim]"
15482,hexahydro,hexahydro 135 trinitroso 135 triazin,0.50,"[h, e, x, a, h, y, d, r, o]","[h, e, x, a, h, y, d, r, o, , 1, 3, 5, , t, ...",[hexahydro],"[hexahydro, 135, trinitroso, 135, triazin]"
2933,based method,method bas,0.75,"[b, a, s, e, d, , m, e, t, h, o, d]","[m, e, t, h, o, d, , b, a, s, e, d]","[based, method]","[method, bas]"
29059,saw sensor,sensor,0.50,"[s, a, w, , s, e, n, s, o, r]","[s, e, n, s, o, r]","[saw, sensor]",[sensor]
12093,expandable intralumin,expandable muscl,0.00,"[e, x, p, a, n, d, a, b, l, e, , i, n, t, r, ...","[e, x, p, a, n, d, a, b, l, e, , m, u, s, c, ...","[expandable, intralumin]","[expandable, muscl]"


### İki Metin arası Benzerlik Ölçümleri

#### 1-Cosine Similarity
cosinus benzerliği kullanarak iki vektör arasındaki benzerliği hesaplayan bir işlemi gerçekleştirir. İşte kodun çalışma mantığı:

from scipy import spatial ile scipy kütüphanesinden uzaklık hesaplamak için gerekli olan cosine fonksiyonunu içe aktarıyoruz.

cos_sim_vals adında boş bir liste oluşturuyoruz, bu liste, her bir "anchor" ve "target" çifti için hesaplanan cosinus benzerlik değerlerini saklamak için kullanılacaktır.

Bir döngü başlatıyoruz, for a, t in zip(anchor_tf, target_tf) ile "anchor_tf" ve "target_tf" adlı iki liste üzerinde dönüyoruz. Bu listeler, "anchor" ve "target" metinlerinin TF-IDF (Term Frequency-Inverse Document Frequency) özellikleri kullanılarak dönüştürülmüş vektörlerdir.

Her çift için cosinus benzerliği hesaplıyoruz. spatial.distance.cosine(a, t) ile iki vektör arasındaki cosinus benzerliği hesaplanır. Benzerlik değeri 1'e yakınsa, vektörler birbirine çok benzerdir.

Hesaplanan benzerlik değeri, cos_sim_vals listesine eklenir.

Sonuç olarak, bu kod, "anchor" ve "target" metinlerinin TF-IDF vektörleri arasındaki cosinus benzerliğini hesaplar ve bu benzerlik değerlerini bir liste içinde saklar. Bu değerler daha sonra başka amaçlar için kullanılabilir, özellikle metin benzerliği değerlendirmeleri veya metin sınıflandırma işlemleri için kullanışlı olabilir.

In [21]:
from scipy import spatial

cos_sim_vals  = []

for a, t in zip(anchor_tf, target_tf):
    similarity = 1 - spatial.distance.cosine(a, t)
    cos_sim_vals.append(similarity)

In [22]:
train_df["Cosine Similarity"] = cos_sim_vals

In [23]:
train_df.head()

,anchor,target,score,lemmatize_a,lemmatize_t,tokenized_a,tokenized_t,Cosine Similarity
0,abat,abatement pollut,0.50,"[a, b, a, t, e, m, e, n, t]","[a, b, a, t, e, m, e, n, t, , p, o, l, l, u, ...",[abat],"[abatement, pollut]",0.616771
1,abat,act ab,0.75,"[a, b, a, t, e, m, e, n, t]","[a, c, t, , a, b, a, t, i, n, g]",[abat],"[act, ab]",0.894857
2,abat,active catalyst,0.25,"[a, b, a, t, e, m, e, n, t]","[a, c, t, i, v, e, , c, a, t, a, l, y, s, t]",[abat],"[active, catalyst]",0.494009
3,abat,eliminating process,0.50,"[a, b, a, t, e, m, e, n, t]","[e, l, i, m, i, n, a, t, i, n, g, , p, r, o, ...",[abat],"[eliminating, process]",0.161542
4,abat,forest region,0.00,"[a, b, a, t, e, m, e, n, t]","[f, o, r, e, s, t, , r, e, g, i, o, n]",[abat],"[forest, region]",0.071977


#### 2-Levenshtein mesafesi
Levenshtein oranını hesaplayan bir Python işlemi gerçekleştirir. İşlevleri şu şekildedir:

İlk olarak, python-Levenshtein adlı bir Python paketini yüklemek için !pip install python-Levenshtein komutunu kullanır. Bu paket, Levenshtein mesafesi ve oranını hesaplama işlemleri için kullanılır.

Daha sonra Levenshtein paketinden distance ve ratio işlevlerini içe aktarır.

Ardından, bir döngü içinde data adlı veri çerçevesinin "anchor" ve "target" sütunları arasında dolaşarak her iki metin arasındaki Levenshtein oranını hesaplar. Levenshtein oranı, iki metin arasındaki benzerliği ölçer ve bir değer döndürür. Bu değer, 0 ile 1 arasında bir sürekli sayıdır, 1'e yaklaştıkça iki metin birbirine daha yakın demektir.

Levenshtein oranları, lev_sim_vals adlı bir liste içinde saklanır.

Son olarak, bu Levenshtein oranlarını "Levenshtein Ratio" adlı bir yeni sütuna ekler. Bu sütun, iki metin arasındaki Levenshtein benzerliği ölçüsünü içerir.

Bu kod parçası, iki metin arasındaki benzerliği hesaplamak ve sonuçları bir veri çerçevesine eklemek için kullanılır. Levenshtein mesafesi ve oranı, metin benzerliği analizlerinde ve özellikle yazım denetimi uygulamalarında sıklıkla kullanılır.

In [24]:
#pip install Levenshtein

In [25]:
from Levenshtein import distance as lev
from Levenshtein import ratio

In [26]:
lev_sim_vals  = []

for a, t in zip(train_df["anchor"], train_df["target"]):
    lev_similarity = ratio(a, t)
    lev_sim_vals.append(lev_similarity)

In [27]:
train_df["Levenshtein Ratio"] = lev_sim_vals

In [28]:
train_df.head(3)

,anchor,target,score,lemmatize_a,lemmatize_t,tokenized_a,tokenized_t,Cosine Similarity,Levenshtein Ratio
0,abat,abatement pollut,0.50,"[a, b, a, t, e, m, e, n, t]","[a, b, a, t, e, m, e, n, t, , p, o, l, l, u, ...",[abat],"[abatement, pollut]",0.616771,0.400000
1,abat,act ab,0.75,"[a, b, a, t, e, m, e, n, t]","[a, c, t, , a, b, a, t, i, n, g]",[abat],"[act, ab]",0.894857,0.400000
2,abat,active catalyst,0.25,"[a, b, a, t, e, m, e, n, t]","[a, c, t, i, v, e, , c, a, t, a, l, y, s, t]",[abat],"[active, catalyst]",0.494009,0.315789


#### 3-Jaccard benzerlik skoru
İki küme veya dizgi arasındaki benzerliği ölçmek için kullanılan bir ölçüdür. Temel olarak, Jaccard benzerliği, iki küme veya dizgi arasındaki ortak öğelerin sayısını, bu iki küme veya dizgi arasındaki toplam farklı öğelerin sayısına bölerek hesaplanır. Bu skor, iki küme veya dizgi arasındaki benzerlik derecesini yüzde olarak ifade eder.

Jaccard benzerlik skoru şu şekilde hesaplanır:

1. İki küme veya dizgi düşünün: A ve B.
2. Jaccard benzerlik skoru hesaplamak için önce A ve B'nin kesişim kümesini (yani, A ve B'de aynı anda bulunan öğelerin kümesini) ve birleşim kümesini (yani, A ve B'nin tüm öğelerinin birleşimini) hesaplayın.
3. Jaccard benzerlik skoru, kesişim kümesinin büyüklüğünü (ortak öğelerin sayısı) birleşim kümesinin büyüklüğüne (farklı öğelerin sayısı) böler.
4. Sonuç, 0 ile 1 arasında bir değer alır. 0, hiçbir ortak öğe olmadığını, 1 ise her iki küme veya dizgi arasında tam bir örtüşme olduğunu gösterir.

Jaccard benzerlik skoru, metin madenciliği, kümeleme analizi, öneri sistemleri ve çeşitli diğer uygulamalarda kullanılır. İki metin veya öğe arasındaki benzerliği yüzde olarak ifade ederek, bu skor sayesinde iki öğenin benzerliği hakkında bilgi edinilebilir.

In [29]:
#pip install distance

In [30]:
import distance
distance.jaccard("decide", "resize")

0.7142857142857143

In [31]:
jac_sim_vals  = []

for a, t in zip(train_df["anchor"], train_df["target"]):
    jac_sim = distance.jaccard(a, t)
    jac_sim_vals.append(jac_sim)

In [32]:
train_df["Jaccard Similarity"] = jac_sim_vals

In [33]:
train_df.tail(3)

,anchor,target,score,lemmatize_a,lemmatize_t,tokenized_a,tokenized_t,Cosine Similarity,Levenshtein Ratio,Jaccard Similarity
36470,wood articl,wooden handl,0.50,"[w, o, o, d, , a, r, t, i, c, l, e]","[w, o, o, d, e, n, , h, a, n, d, l, e]","[wood, articl]","[wooden, handl]",0.698958,0.608696,0.538462
36471,wood articl,wooden materi,0.75,"[w, o, o, d, , a, r, t, i, c, l, e]","[w, o, o, d, e, n, , m, a, t, e, r, i, a, l]","[wood, articl]","[wooden, materi]",0.789996,0.666667,0.384615
36472,wood articl,wooden substr,0.50,"[w, o, o, d, , a, r, t, i, c, l, e]","[w, o, o, d, e, n, , s, u, b, s, t, r, a, t, e]","[wood, articl]","[wooden, substr]",0.628353,0.500000,0.600000


#### 4-Öklidyen Mesafesi
İki nokta veya vektör arasındaki öklidyen uzaklığı (Euclidean distance) hesaplayan ve bu uzaklıkları belirli bir aralığa (genellikle 0 ile 1 arasına) normalize eden işlemleri gerçekleştirir. Öklidyen uzaklık, n-boyutlu uzayda iki nokta arasındaki en kısa mesafeyi ölçmek için kullanılır. İki vektör arasındaki Öklidyen uzaklık, L2 normunu hesaplayarak bulunur.

Aşağıdaki işlemleri gerçekleştirir:

1. İlk olarak, iki metin öğesini temsil eden vektörlerin (anchor_tf ve target_tf) arasındaki Öklidyen uzaklığı hesaplar ve bunları euc_sim_vals adlı bir liste içinde saklar.

2. Ardından, hesaplanan Öklidyen uzaklıkları bir diziye dönüştürür.

3. Daha sonra, uzaklıkları 0 ile 1 arasında normalize etmek için MinMaxScaler kullanır. Bu, uzaklıkları veri madenciliği ve benzeri analizlerde daha anlamlı hale getirir.

4. Son olarak, normalize edilmiş uzaklıkları "Normalized Eucliden" adlı bir sütunda veri çerçevesine ekler.

Bu işlem, iki metin veya öğe arasındaki benzerliği ölçmek için Öklidyen uzaklığı kullanarak yapılan bir benzerlik hesaplama işlemidir. Benzerlik ölçüsü daha sonra veri madenciliği veya diğer analizler için kullanılabilir.

In [34]:
from sklearn.preprocessing import MinMaxScaler

In [35]:
euc_sim_vals = []
for a, t in zip(anchor_tf, target_tf):
    dist = np.linalg.norm(a-t)
    euc_sim_vals.append(dist)

In [36]:
len(euc_sim_vals)

36473

In [37]:
euc_sim_vals = np.array(euc_sim_vals)

euc_sim_vals listesi, önceki kod parçasında Öklidyen uzaklıkların hesaplanması sırasında bu uzaklıkları saklamak amacıyla oluşturulmuştu. Öklidyen uzaklıkların hesaplanmasının ardından bu liste, bir liste objesi olarak bulunuyordu. Ancak bazen bu verileri daha etkili bir şekilde işlemek ve matematiksel işlemler yapmak için NumPy dizileri kullanmak daha iyidir.

Bu nedenle, np.array(euc_sim_vals) kodu, euc_sim_vals listesini bir NumPy dizisine dönüştürür. NumPy dizileri, matematiksel işlemleri hızlandırmak ve daha fazla işlevselliği kolayca kullanabilmek için kullanışlıdır. Bu nedenle, genellikle veri analizi veya bilimsel hesaplama uygulamalarında tercih edilirler.

In [38]:
mms = MinMaxScaler()
normalized_euc= mms.fit_transform(euc_sim_vals.reshape(-1,1))

In [39]:
len(normalized_euc)

36473

In [40]:
train_df["Normalized Eucliden"] = normalized_euc

In [41]:
train_df.sample(3)

,anchor,target,score,lemmatize_a,lemmatize_t,tokenized_a,tokenized_t,Cosine Similarity,Levenshtein Ratio,Jaccard Similarity,Normalized Eucliden
24416,pillar,pillared clay,0.5,"[p, i, l, l, a, r, i, n, g]","[p, i, l, l, a, r, e, d, , c, l, a, y]",[pillar],"[pillared, clay]",0.798095,0.631579,0.500000,0.449338
5637,clocked storag,digital storage regist,0.5,"[c, l, o, c, k, e, d, , s, t, o, r, a, g, e]","[d, i, g, i, t, a, l, , s, t, o, r, a, g, e, ...","[clocked, storag]","[digital, storage, regist]",0.600207,0.444444,0.230769,0.632292
18085,length light,reflecting surface dist,0.5,"[l, e, n, g, t, h, , l, i, g, h, t]","[r, e, f, l, e, c, t, i, n, g, , s, u, r, f, ...","[length, light]","[reflecting, surface, dist]",0.404570,0.400000,0.533333,0.771641


In [42]:
# DeepLearning
train_df.columns

Index(['anchor', 'target', 'score', 'lemmatize_a', 'lemmatize_t',
       'tokenized_a', 'tokenized_t', 'Cosine Similarity', 'Levenshtein Ratio',
       'Jaccard Similarity', 'Normalized Eucliden'],
      dtype='object')

In [43]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   anchor               36473 non-null  object 
 1   target               36473 non-null  object 
 2   score                36473 non-null  float64
 3   lemmatize_a          36473 non-null  object 
 4   lemmatize_t          36473 non-null  object 
 5   tokenized_a          36473 non-null  object 
 6   tokenized_t          36473 non-null  object 
 7   Cosine Similarity    36473 non-null  float64
 8   Levenshtein Ratio    36473 non-null  float64
 9   Jaccard Similarity   36473 non-null  float64
 10  Normalized Eucliden  36473 non-null  float64
dtypes: float64(5), object(6)
memory usage: 3.1+ MB


In [44]:
x = pd.concat([train_df[['Cosine Similarity', 'Levenshtein Ratio', 'Jaccard Similarity', 'Normalized Eucliden']],
               pd.get_dummies(train_df[['anchor', 'target']])], axis=1)
y = train_df['score']

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten

In [46]:
train_df["score"].value_counts()

0.50    12300
0.25    11519
0.00     7471
0.75     4029
1.00     1154
Name: score, dtype: int64

In [47]:
from keras.utils import to_categorical
y_encoded = to_categorical(y, num_classes=5)

In [48]:
model = Sequential()
model.add(Dense(6, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(5, activation='softmax'))  # Beş düğüm (beş sınıf için)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x, y_encoded, epochs=15, batch_size=10, validation_split=0.20, verbose=1)

In [ ]:
model.summary()

In [ ]:
model.evaluate(x,y)